In [102]:
import logging
import re
import os
# os.chdir('../code/')
import random

import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from tqdm.notebook import tqdm

import json
import itertools

import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [4]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')



In [5]:
# TODO: deal with data path

true_folder = '../thai_data/'

model_domain = 'laptop14-rest14' # rest14-laptop14
is_tokenizer = 'data_tokenize' # non_tokenize 
pred_folder = '../data_predict/text-to-label'

In [6]:
true_file_path = '../thai_data/rest14/th/googletrans/test_aste_processed.txt'
pred_file_path = '../data_predict/text-to-label/laptop14-rest14/output_predict/non_tokenize/rest14/th/googletrans/test_aste_processed.txt'

## SET: file path

## Process `true_file` 

In [7]:
# for true_label file

with open(true_file_path, 'r') as file:
    lines = file.readlines()
    true_list = [line.replace('\n','') for line in lines]

true_df = pd.DataFrame(true_list,columns=['true_raw'])
true_df.head(3)


,true_raw
0,ขนมปังเป็นรอยด้านบนเช่นกัน ===> <pos> ขนมปัง <...
1,ฉันต้องบอกว่าพวกเขามีหนึ่งในเวลาส่งมอบที่เร็วท...
2,อาหารสดและร้อนพร้อมที่จะกินเสมอ! ===> <pos> อา...


In [8]:
true_df[['input_text', 'true_labels_text']] = true_df['true_raw'].apply(lambda x: pd.Series(x.split(' ===> ')))
true_df


,true_raw,input_text,true_labels_text
0,ขนมปังเป็นรอยด้านบนเช่นกัน ===> <pos> ขนมปัง <...,ขนมปังเป็นรอยด้านบนเช่นกัน,<pos> ขนมปัง <opinion> รอยบากด้านบน
1,ฉันต้องบอกว่าพวกเขามีหนึ่งในเวลาส่งมอบที่เร็วท...,ฉันต้องบอกว่าพวกเขามีหนึ่งในเวลาส่งมอบที่เร็วท...,<pos> เวลาจัดส่ง <opinion> เร็วที่สุด
2,อาหารสดและร้อนพร้อมที่จะกินเสมอ! ===> <pos> อา...,อาหารสดและร้อนพร้อมที่จะกินเสมอ!,<pos> อาหาร <opinion> สด <pos> อาหาร <opinion>...
3,ฉันพูดถึงว่ากาแฟนั้นโดดเด่นหรือไม่? ===> <pos>...,ฉันพูดถึงว่ากาแฟนั้นโดดเด่นหรือไม่?,<pos> กาแฟ <opinion> โดดเด่น
4,แน่นอนว่าไม่ใช่ซูชิที่ดีที่สุดในนิวยอร์กอย่างไ...,แน่นอนว่าไม่ใช่ซูชิที่ดีที่สุดในนิวยอร์กอย่างไ...,<pos> สถานที่ <opinion> ทำความสะอาด
...,...,...,...
487,บาร์เทนเดอร์ชาวไอริชแสนหวานมักจะมีความสุขและสา...,บาร์เทนเดอร์ชาวไอริชแสนหวานมักจะมีความสุขและสา...,<pos> บาร์เทนเดอร์ <opinion> หวาน <pos> บาร์เท...
488,เป็นการดีที่ได้ไปดื่มถ้าคุณไม่อยากเมาเพราะคุณจ...,เป็นการดีที่ได้ไปดื่มถ้าคุณไม่อยากเมาเพราะคุณจ...,<neg> บริการ <opinion> แย่ <neu> เครื่องดื่ม <...
489,อย่างไรก็ตามเจ้าของเป็นของปลอม ===> <neg> เจ้า...,อย่างไรก็ตามเจ้าของเป็นของปลอม,<neg> เจ้าของ <opinion> ปลอม
490,เจ้าของเป็นที่น่าพอใจและสนุกสนาน ===> <pos> เจ...,เจ้าของเป็นที่น่าพอใจและสนุกสนาน,<pos> เจ้าของ <opinion> เพลิดเพลิน <pos> เจ้าข...


In [9]:
def find_labels(text_label:str):
    enth = r'[\w\u0E01-\u0E5B\(\) ]'
    label_pattern = re.compile(r'<(pos|neg|neu)> ({0}+) <opinion> ({0}+)'.format(enth))

    
    labels = re.findall(label_pattern, text_label)
    return labels

In [10]:
true_df['true_labels'] = true_df['true_labels_text'].apply(lambda x: find_labels(x))
true_df

,true_raw,input_text,true_labels_text,true_labels
0,ขนมปังเป็นรอยด้านบนเช่นกัน ===> <pos> ขนมปัง <...,ขนมปังเป็นรอยด้านบนเช่นกัน,<pos> ขนมปัง <opinion> รอยบากด้านบน,"[(pos, ขนมปัง, รอยบากด้านบน )]"
1,ฉันต้องบอกว่าพวกเขามีหนึ่งในเวลาส่งมอบที่เร็วท...,ฉันต้องบอกว่าพวกเขามีหนึ่งในเวลาส่งมอบที่เร็วท...,<pos> เวลาจัดส่ง <opinion> เร็วที่สุด,"[(pos, เวลาจัดส่ง, เร็วที่สุด )]"
2,อาหารสดและร้อนพร้อมที่จะกินเสมอ! ===> <pos> อา...,อาหารสดและร้อนพร้อมที่จะกินเสมอ!,<pos> อาหาร <opinion> สด <pos> อาหาร <opinion>...,"[(pos, อาหาร, สด ), (pos, อาหาร, ร้อน )]"
3,ฉันพูดถึงว่ากาแฟนั้นโดดเด่นหรือไม่? ===> <pos>...,ฉันพูดถึงว่ากาแฟนั้นโดดเด่นหรือไม่?,<pos> กาแฟ <opinion> โดดเด่น,"[(pos, กาแฟ, โดดเด่น )]"
4,แน่นอนว่าไม่ใช่ซูชิที่ดีที่สุดในนิวยอร์กอย่างไ...,แน่นอนว่าไม่ใช่ซูชิที่ดีที่สุดในนิวยอร์กอย่างไ...,<pos> สถานที่ <opinion> ทำความสะอาด,"[(pos, สถานที่, ทำความสะอาด )]"
...,...,...,...,...
487,บาร์เทนเดอร์ชาวไอริชแสนหวานมักจะมีความสุขและสา...,บาร์เทนเดอร์ชาวไอริชแสนหวานมักจะมีความสุขและสา...,<pos> บาร์เทนเดอร์ <opinion> หวาน <pos> บาร์เท...,"[(pos, บาร์เทนเดอร์, หวาน ), (pos, บาร์เทนเดอร..."
488,เป็นการดีที่ได้ไปดื่มถ้าคุณไม่อยากเมาเพราะคุณจ...,เป็นการดีที่ได้ไปดื่มถ้าคุณไม่อยากเมาเพราะคุณจ...,<neg> บริการ <opinion> แย่ <neu> เครื่องดื่ม <...,"[(neg, บริการ, แย่ ), (neu, เครื่องดื่ม, ดี )]"
489,อย่างไรก็ตามเจ้าของเป็นของปลอม ===> <neg> เจ้า...,อย่างไรก็ตามเจ้าของเป็นของปลอม,<neg> เจ้าของ <opinion> ปลอม,"[(neg, เจ้าของ, ปลอม )]"
490,เจ้าของเป็นที่น่าพอใจและสนุกสนาน ===> <pos> เจ...,เจ้าของเป็นที่น่าพอใจและสนุกสนาน,<pos> เจ้าของ <opinion> เพลิดเพลิน <pos> เจ้าข...,"[(pos, เจ้าของ, เพลิดเพลิน ), (pos, เจ้าของ, ค..."


## Process `infer_file`

In [11]:
true_df.columns

Index(['true_raw', 'input_text', 'true_labels_text', 'true_labels'], dtype='object')

In [12]:
# for true_label file

with open(pred_file_path, 'r') as file:
    lines = file.readlines()
    pred_list = [line.replace('\n','') for line in lines]
    pred_list = [line for line in pred_list if line != '']


pred_df = pd.DataFrame({'input_text': pred_list[::2], 'pred_labels_text': pred_list[1::2]})
pred_df.head(3)


,input_text,pred_labels_text
0,Input text: ขนมปังเป็นรอยด้านบนเช่นกัน,Predicted (generated) text: <pos> ขนมปังซึ่งเป...
1,Input text: ฉันต้องบอกว่าพวกเขามีหนึ่งในเวลาส่...,Predicted (generated) text: <neg> เวลาส่งมอบ <...
2,Input text: อาหารสดและร้อนพร้อมที่จะกินเสมอ!,Predicted (generated) text: <pos> อาหาร <opini...


In [13]:
pred_df['input_text'] = pred_df['input_text'].apply(lambda x: x.replace('Input text: ',''))
pred_df['pred_labels_text'] = pred_df['pred_labels_text'].apply(lambda x: x.replace('Predicted (generated) text: ',''))

In [14]:
pred_df.head(3)

,input_text,pred_labels_text
0,ขนมปังเป็นรอยด้านบนเช่นกัน,<pos> ขนมปังซึ่งเป็นรอยด้านบน <opinion> ขนมปัง...
1,ฉันต้องบอกว่าพวกเขามีหนึ่งในเวลาส่งมอบที่เร็วท...,<neg> เวลาส่งมอบ <opinion> ครั้งแรก
2,อาหารสดและร้อนพร้อมที่จะกินเสมอ!,<pos> อาหาร <opinion> สด <pos> อาหาร <opinion>...


In [15]:
pred_df['pred_labels'] = pred_df['pred_labels_text'].apply(lambda x: find_labels(x))
pred_df

,input_text,pred_labels_text,pred_labels
0,ขนมปังเป็นรอยด้านบนเช่นกัน,<pos> ขนมปังซึ่งเป็นรอยด้านบน <opinion> ขนมปัง...,"[(pos, ขนมปังซึ่งเป็นรอยด้านบน, ขนมปังซึ่งเป็น..."
1,ฉันต้องบอกว่าพวกเขามีหนึ่งในเวลาส่งมอบที่เร็วท...,<neg> เวลาส่งมอบ <opinion> ครั้งแรก,"[(neg, เวลาส่งมอบ, ครั้งแรก)]"
2,อาหารสดและร้อนพร้อมที่จะกินเสมอ!,<pos> อาหาร <opinion> สด <pos> อาหาร <opinion>...,"[(pos, อาหาร, สด ), (pos, อาหาร, ร้อน)]"
3,ฉันพูดถึงว่ากาแฟนั้นโดดเด่นหรือไม่?,<pos> กาแฟ <opinion> โดดเด่น,"[(pos, กาแฟ, โดดเด่น)]"
4,แน่นอนว่าไม่ใช่ซูชิที่ดีที่สุดในนิวยอร์กอย่างไ...,<neg> ซูชิ <opinion> ไม่ใช่ <neg> ซูชิ <opinio...,"[(neg, ซูชิ, ไม่ใช่ ), (neg, ซูชิ, ปลอดเชื้อ)]"
...,...,...,...
487,บาร์เทนเดอร์ชาวไอริชแสนหวานมักจะมีความสุขและสา...,<pos> บาร์เทนเดอร์ <opinion> มีความสุข <pos> บ...,"[(pos, บาร์เทนเดอร์, มีความสุข ), (pos, บาร์เท..."
488,เป็นการดีที่ได้ไปดื่มถ้าคุณไม่อยากเมาเพราะคุณจ...,<pos> service <opinion> ดี,"[(pos, service, ดี)]"
489,อย่างไรก็ตามเจ้าของเป็นของปลอม,<neg> เจ้าของ <opinion> ปลอม,"[(neg, เจ้าของ, ปลอม)]"
490,เจ้าของเป็นที่น่าพอใจและสนุกสนาน,<pos> เจ้าของ <opinion> ที่น่าพอใจ <pos> keys ...,"[(pos, เจ้าของ, ที่น่าพอใจ ), (pos, keys, สนุก..."


## Merge df true & pred

In [16]:
# pd.merge(true_df, pred_df, how='inner')
true_df['true_n_labels'] = true_df['true_labels'].apply(lambda x: len(x))
pred_df['pred_n_labels'] = pred_df['pred_labels'].apply(lambda x: len(x))
# true_df.merge(pred_df, how='inner', on='input_text')

## check labels

In [17]:
df = true_df.merge(pred_df, how='inner', on='input_text')
df.columns

Index(['true_raw', 'input_text', 'true_labels_text', 'true_labels',
       'true_n_labels', 'pred_labels_text', 'pred_labels', 'pred_n_labels'],
      dtype='object')

In [18]:
df[['true_labels','pred_labels', 'true_n_labels','pred_n_labels']]

,true_labels,pred_labels,true_n_labels,pred_n_labels
0,"[(pos, ขนมปัง, รอยบากด้านบน )]","[(pos, ขนมปังซึ่งเป็นรอยด้านบน, ขนมปังซึ่งเป็น...",1,1
1,"[(pos, เวลาจัดส่ง, เร็วที่สุด )]","[(neg, เวลาส่งมอบ, ครั้งแรก)]",1,1
2,"[(pos, อาหาร, สด ), (pos, อาหาร, ร้อน )]","[(pos, อาหาร, สด ), (pos, อาหาร, ร้อน)]",2,2
3,"[(pos, กาแฟ, โดดเด่น )]","[(pos, กาแฟ, โดดเด่น)]",1,1
4,"[(pos, สถานที่, ทำความสะอาด )]","[(neg, ซูชิ, ไม่ใช่ ), (neg, ซูชิ, ปลอดเชื้อ)]",1,2
...,...,...,...,...
330,"[(neg, บริการ, ช้าอย่างรุนแรง ), (neu, อาหาร, ...","[(neg, Service, ปานกลาง ), (neg, และบริการ, sl...",2,2
331,"[(neg, บริการ, แย่ ), (neu, เครื่องดื่ม, ดี )]","[(pos, service, ดี)]",2,1
332,"[(neg, เจ้าของ, ปลอม )]","[(neg, เจ้าของ, ปลอม)]",1,1
333,"[(pos, เจ้าของ, เพลิดเพลิน ), (pos, เจ้าของ, ค...","[(pos, เจ้าของ, ที่น่าพอใจ ), (pos, keys, สนุก...",2,2


In [21]:
# get_score_me5(
df[['true_labels','pred_labels']]

,true_labels,pred_labels
0,"[(pos, ขนมปัง, รอยบากด้านบน )]","[(pos, ขนมปังซึ่งเป็นรอยด้านบน, ขนมปังซึ่งเป็น..."
1,"[(pos, เวลาจัดส่ง, เร็วที่สุด )]","[(neg, เวลาส่งมอบ, ครั้งแรก)]"
2,"[(pos, อาหาร, สด ), (pos, อาหาร, ร้อน )]","[(pos, อาหาร, สด ), (pos, อาหาร, ร้อน)]"
3,"[(pos, กาแฟ, โดดเด่น )]","[(pos, กาแฟ, โดดเด่น)]"
4,"[(pos, สถานที่, ทำความสะอาด )]","[(neg, ซูชิ, ไม่ใช่ ), (neg, ซูชิ, ปลอดเชื้อ)]"
...,...,...
330,"[(neg, บริการ, ช้าอย่างรุนแรง ), (neu, อาหาร, ...","[(neg, Service, ปานกลาง ), (neg, และบริการ, sl..."
331,"[(neg, บริการ, แย่ ), (neu, เครื่องดื่ม, ดี )]","[(pos, service, ดี)]"
332,"[(neg, เจ้าของ, ปลอม )]","[(neg, เจ้าของ, ปลอม)]"
333,"[(pos, เจ้าของ, เพลิดเพลิน ), (pos, เจ้าของ, ค...","[(pos, เจ้าของ, ที่น่าพอใจ ), (pos, keys, สนุก..."


In [96]:
THESHOLD_ASPECT = 0.9
THESHOLD_OPINION = 0.9

CORRECT_RATIO = 0.8

def is_this_pred_correct(pred: tuple, all_true_label: list):
   
   
#    return [check_label_pair(pred, t) for t in all_true_label]
   return any([check_label_pair(pred, t) for t in all_true_label])

def check_all_pred(pred_list: list, true_list: list):
   
    check_list = []
    for p in pred_list:
    #    print(f'{p} in {true_list} --> {is_this_pred_correct(p, true_list)}')
       check_list.append(is_this_pred_correct(p, true_list))

    return check_list
    



def check_label_pair(x, y):
    x_s, x_a, x_o = x   # true label
    y_s, y_a, y_o = y   # pred label

    # return f'do_something with {a} and {b}'

    score = get_score_me5
    # print(f'{score.__name__} of aspect: {x_a} and {y_a}')
    # print(f'{score.__name__} of opinion: {x_o} and {y_o}')
    # return x_s == y_s, \
    #     score(x_a, y_a) > THESHOLD_ASPECT, \
    #     score(x_o, y_o) > THESHOLD_OPINION
    return x_s == y_s and \
        score(x_a, y_a) > THESHOLD_ASPECT and \
        score(x_o, y_o) > THESHOLD_OPINION


def get_similar_muse(source_text:str, target_text: str):

    return np.random.uniform(0, 1.0)

def get_score_me5(source_text:str, target_text: str):
  input_texts = [source_text, target_text]
  batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
  outputs = model(**batch_dict)
  embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
  embeddings = F.normalize(embeddings, p=2, dim=1)
  scores = (embeddings[:1] @ embeddings[1:].T)
  return scores.item()



In [97]:
index = 333
# check_label_pair(df.true_labels[index][0], df.pred_labels[index][0])

is_this_pred_correct(df.pred_labels[index][0], df.true_labels[index])
check_all_pred(df.pred_labels[index], df.true_labels[index])

[True, False]

In [99]:
df.columns

Index(['true_raw', 'input_text', 'true_labels_text', 'true_labels',
       'true_n_labels', 'pred_labels_text', 'pred_labels', 'pred_n_labels'],
      dtype='object')

In [106]:
tqdm.pandas(desc='check all pred labels')
df['check_labels'] = df.progress_apply(lambda x: check_all_pred(x.pred_labels, x.true_labels),axis=1)

check all pred labels:   0%|          | 0/335 [00:00<?, ?it/s]

In [108]:
df['check_labels'].apply(lambda x: sum(x) / len(x))
# df['percent_correct'] 

ZeroDivisionError: division by zero